In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import img_to_array, load_img

/home/cocos/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [3]:
def get_dataset_mnist():
    df = pd.read_csv('./mnist.csv')
    y = df['label'].values
    d=[]
    for i in range(df.shape[1]-1):
        d.append(df[str(i)])

    X = np.array(d).transpose()
    
    return X, y

In [4]:
def split_data(X, y, shape):
    classes = 10
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train.shape = (len(X_train), shape[0], shape[1], shape[2])
    X_test.shape = (len(X_test), shape[0], shape[1], shape[2])
    y_train = tf.keras.utils.to_categorical(y_train, classes)
    y_test = tf.keras.utils.to_categorical(y_test, classes)

    return X_train, X_test, y_train, y_test

In [9]:
def CNN1(activation, X_train, X_test, y_train, y_test): 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=activation, 
                                     input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=activation))
    model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=activation))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation=activation))
    model.add(tf.keras.layers.Dense(64, activation=activation))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

    score = model.evaluate(X_test, y_test)
    print('loss:', score[0])
    print('accuracy:', score[1])

In [ ]:
X, y = get_dataset_mnist()
X_train, X_test, y_train, y_test = split_data(X, y, (28, 28, 1))
CNN1('relu', X_train, X_test, y_train, y_test)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 3s 430us/sample - loss: 0.6060 - acc: 0.8425 - val_loss: 0.1960 - val_acc: 0.9365
Epoch 2/5
8000/8000 [==============================] - 4s 449us/sample - loss: 0.1011 - acc: 0.9701 - val_loss: 0.2102 - val_acc: 0.9395
Epoch 3/5
8000/8000 [==============================] - 3s 422us/sample - loss: 0.0455 - acc: 0.9856 - val_loss: 0.1372 - val_acc: 0.9575
Epoch 4/5
8000/8000 [==============================] - 3s 427us/sample - loss: 0.0257 - acc: 0.9915 - val_loss: 0.1468 - val_acc: 0.9620
Epoch 5/5
7456/8000 [==========================>...] - ETA: 0s - loss: 0.0220 - acc: 0.9937

In [1]:
CNN1('sigmoid', X_train, X_test, y_train, y_test)

NameError: name 'CNN1' is not defined

In [ ]:
CNN1('tanh', X_train, X_test, y_train, y_test)

### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [154]:
def get_dataset_notMnist():
    X, y = [], []
    letters = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9}
    p = Path('./notMNIST/notMNIST_small/')
    dirs = [x for x in p.iterdir() if x.is_dir()]

    for el in dirs:
        p = Path(el)
        for x in p.iterdir():
            if x.is_file():
                img = load_img(x)
                img_array = img_to_array(img)
                X.append(img_array)
                path = str(el)
                letter = path[len(path)-1:len(path)]
                num = letters[letter]
                y.append(num)
    X = np.array(X)
    y = np.array(y)
    return X, y

In [235]:
def CNN2(activation, X_train, X_test, y_train, y_test): 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(28,28,3)))

    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.1))
    
    model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Flatten())   
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
   
    model.fit(X_train, y_train, batch_size=128, epochs=15, validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test)
    print('loss:', score[0])
    print('accuracy:', score[1])

In [236]:
X, y = get_dataset_notMnist()
X_train, X_test, y_train, y_test = split_data(X, y, (28, 28, 3))
CNN2('relu', X_train, X_test, y_train, y_test)

Train on 14979 samples, validate on 3745 samples
Epoch 1/15
14979/14979 [==============================] - 10s 651us/sample - loss: 13.0018 - acc: 0.1897 - val_loss: 11.5843 - val_acc: 0.2774
Epoch 2/15
14979/14979 [==============================] - 9s 581us/sample - loss: 10.2165 - acc: 0.3596 - val_loss: 8.1006 - val_acc: 0.4935
Epoch 3/15
14979/14979 [==============================] - 9s 610us/sample - loss: 8.6201 - acc: 0.4604 - val_loss: 7.9058 - val_acc: 0.5052
Epoch 4/15
14979/14979 [==============================] - 10s 646us/sample - loss: 7.5223 - acc: 0.5259 - val_loss: 6.4878 - val_acc: 0.5933
Epoch 5/15
14979/14979 [==============================] - 10s 639us/sample - loss: 6.6756 - acc: 0.5791 - val_loss: 5.4712 - val_acc: 0.6510
Epoch 6/15
14979/14979 [==============================] - 10s 634us/sample - loss: 5.6605 - acc: 0.6394 - val_loss: 5.0371 - val_acc: 0.6804
Epoch 7/15
14979/14979 [==============================] - 10s 640us/sample - loss: 5.2228 - acc: 0.6671 